# Classification semi-supervisée

/!\ Ce notebook dépend de deux fichiers générés par le notebook précédent.


In [2]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.semi_supervised import SelfTrainingClassifier, LabelSpreading

In [ ]:
# df_train = pd.read_csv('./data/forest-cover-type/train.csv', index_col=0 )
# X_test = pd.read_csv('./data/forest-cover-type/test.csv', index_col=0 )

# # separation des données
# X = df_train.drop(columns=['Cover_Type'])
# y = df_train['Cover_Type']

# # divise les données 
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.75, random_state=42)
# columns_to_keep = ['Wilderness_Area1',
#        'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
#        'Geologic_Zone_1','Climatic_Zone_1', 'Climatic_Zone_2', 'Geologic_Zone_2',
#        'Climatic_Zone_3', 'Geologic_Zone_3', 'Climatic_Zone_4',
#        'Geologic_Zone_4', 'Climatic_Zone_5', 'Geologic_Zone_5',
#        'Climatic_Zone_6', 'Geologic_Zone_6', 'Climatic_Zone_7',
#        'Geologic_Zone_7', 'Climatic_Zone_8', 'Geologic_Zone_8']
# columns_to_norm = [col for col in X.columns if col not in columns_to_keep]

# X_train_scaled = X_train.copy()
# X_val_scaled = X_val.copy()
# X_test_scaled = X_test.copy()

# scaler = StandardScaler()
# # Normalisation sur l'ensemble d'entraînement
# X_train_scaled[columns_to_norm] = scaler.fit_transform(X_train[columns_to_norm])

# # Appliquer la même transformation sur l'ensemble de validation
# X_val_scaled[columns_to_norm] = scaler.transform(X_val[columns_to_norm])


# #données test 
# X_test_scaled[columns_to_norm] = scaler.transform(X_test[columns_to_norm])

## Forêts aléatoires

In [63]:
rf1 = RandomForestClassifier()
rf1.fit(X_train, y_train)

RandomForestClassifier()

In [64]:
predictions = rf1.predict(X_val)
print(X_val.shape)
print(accuracy_score(predictions, y_val))

(75000, 54)
0.8565733333333333


In [65]:
X_train_unlabelled, X_val2, y_train_unlabelled, y_val2 = train_test_split(X_val, y_val, test_size=0.25, random_state=42)

print(
    f"X_train_labelled shape: \t{X_train.shape}",
    f"X_train_unlabelled shape: \t{X_train_unlabelled.shape}",
    f"X_val shape:             \t{X_val2.shape}",
    sep = "\n"
)

X_train_labelled shape: 	(25000, 54)
X_train_unlabelled shape: 	(56250, 54)
X_val shape:             	(18750, 54)


In [ ]:
model = SelfTraining(RandomForestClassifier(), threshold=0.9, n_iter=15)
model.fit(X_train, y_train, X_train_unlabelled, y_val2, X_val2)

In [73]:
# Évaluation des performances
accuracy_score(y_val2, model.predict(X_val2))

/Users/raphaelbordas/miniconda3/envs/deep/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.39525333333333335

In [13]:
# df_train = pd.read_csv('./data/forest-cover-type/train.csv', index_col=0 )
X_test = pd.read_csv('./data/forest-cover-type/test.csv', index_col=0 )

X_select = pd.read_csv("data/forest-cover-type/X_train_for_rf.csv", index_col=0)
X_select_val = pd.read_csv("data/forest-cover-type/X_val_for_rf.csv", index_col=0)
X_test_select = pd.read_csv("data/forest-cover-type/X_test_for_rf.csv", index_col=0)
y_train = pd.read_csv("data/forest-cover-type/y_train_for_rf.csv", index_col=0)
y_val = pd.read_csv("data/forest-cover-type/y_val_for_rf.csv", index_col=0)

# # separation des données
# X = df_train.drop(columns=['Cover_Type'])
# y = df_train['Cover_Type']

# # divise les données 
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

# columns_to_keep = ['Wilderness_Area1',
#        'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
#        'Geologic_Zone_1','Climatic_Zone_1', 'Climatic_Zone_2', 'Geologic_Zone_2',
#        'Climatic_Zone_3', 'Geologic_Zone_3', 'Climatic_Zone_4',
#        'Geologic_Zone_4', 'Climatic_Zone_5', 'Geologic_Zone_5',
#        'Climatic_Zone_6', 'Geologic_Zone_6', 'Climatic_Zone_7',
#        'Geologic_Zone_7', 'Climatic_Zone_8', 'Geologic_Zone_8']

# columns_to_norm = [col for col in X.columns if col not in columns_to_keep]

# X_train_scaled = X_train.copy()
# X_val_scaled = X_val.copy()
# X_test_scaled = X_test.copy()

# scaler = StandardScaler()
# # Normalisation sur l'ensemble d'entraînement
# X_train_scaled[columns_to_norm] = scaler.fit_transform(X_train[columns_to_norm])

# # Appliquer la même transformation sur l'ensemble de validation
# X_val_scaled[columns_to_norm] = scaler.transform(X_val[columns_to_norm])

# #données test 
# X_test_scaled[columns_to_norm] = scaler.transform(X_test[columns_to_norm])

In [5]:
rng = np.random.RandomState(0)

y_rand = rng.rand(y_train.shape[0])
y_30 = np.copy(y_train)
y_30[y_rand < 0.3] = -1
y_70 = np.copy(y_train)
y_70[y_rand < 0.7] = -1
# ls30 = LabelSpreading().fit(X_train_scaled, y_30)
# ls70 = LabelSpreading().fit(X_train_scaled, y_70)

In [6]:
classes = np.array([1, 2, 3, 4, 5, 6, 7])
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights_dict = dict(zip(classes, class_weights))
params = {'max_depth': 50, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250}
# the base classifier for self-training is identical to the SVC
rf_base = RandomForestClassifier(**params, class_weight=class_weights_dict, n_jobs=-1)
rf_base.fit(X_train_scaled, y_train)
preds_base = rf_base.predict(X_val_scaled)
print(accuracy_score(y_val, preds_base))

0.91228


In [ ]:
rf30 = (
    SelfTrainingClassifier(RandomForestClassifier(**params, class_weight=class_weights_dict), threshold=0.85, verbose=True).fit(X_train_scaled, y_30),
    y_30,
    f"Self-training 30% data",
)
rf70 = (
    SelfTrainingClassifier(RandomForestClassifier(**params, class_weight=class_weights_dict), threshold=0.85, verbose=True).fit(X_train_scaled, y_70),
    y_70,
    f"Self-training 50% data",
)

In [9]:
preds30 = rf30[0].predict(X_val_scaled)
print(accuracy_score(y_val, preds30))

preds70 = rf70[0].predict(X_val_scaled)
print(accuracy_score(y_val, preds70))

0.8934
0.8394


In [37]:
classes = np.array([1, 2, 3, 4, 5, 6, 7])
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=pd.concat([y_train, y_val]).Cover_Type)
class_weights_dict = dict(zip(classes, class_weights))
params = {'max_depth': 50, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250}

X_semi_sup = pd.concat([X_select, X_select_val], axis=0)
y_test = pd.Series(- np.ones(X_test_select.shape[0]))
y_all = pd.concat([y_train.Cover_Type, y_val.Cover_Type, y_test], axis=0)

base_cls = RandomForestClassifier(**params, class_weight=class_weights_dict, n_jobs=-1)
self_trainer_rf = SelfTrainingClassifier(base_cls, threshold=0.85, max_iter=10, verbose=True)
self_trainer_rf.fit(pd.concat([X_semi_sup, X_test_select]), y_all)
predictions = self_trainer_rf.predict(X_test_select)

df_pred = pd.DataFrame(predictions, columns=["Cover_Type"]).reset_index().rename(columns={"index": "row_ID"})
display(df_pred.head(15))

df_pred.to_parquet("submission/forest-cover-type/RF_self_training_thres85.parquet", index=False)

End of iteration 1, added 305426 new labels.
End of iteration 2, added 66396 new labels.
End of iteration 3, added 23131 new labels.
End of iteration 4, added 11782 new labels.
End of iteration 5, added 7341 new labels.
End of iteration 6, added 5532 new labels.
End of iteration 7, added 3974 new labels.
End of iteration 8, added 2841 new labels.
End of iteration 9, added 1969 new labels.
End of iteration 10, added 1591 new labels.


,row_ID,Cover_Type
0,0,2.0
1,1,2.0
2,2,3.0
3,3,2.0
4,4,3.0
5,5,2.0
6,6,1.0
7,7,1.0
8,8,2.0
9,9,1.0


In [ ]:
y_test = - np.ones(X_test_scaled.shape[0])
y_all = pd.concat([y_train, y_val, y_test])
self_trainer_rf = SelfTrainingClassifier(RandomForestClassifier(**params, class_weight=class_weights_dict), threshold=0.75, verbose=True)
self_trainer_rf.fit(pd.concat([X_train_scaled, X_val_scaled, X_test_scaled]), y_all)

predictions = self_trainer_rf.predict(X_test_scaled)

df_pred = pd.DataFrame(predictions, columns=["Cover_Type"]).reset_index().rename(columns={"index": "row_ID"})
display(df_pred.head(15))

df_pred.to_parquet("submission/forest-cover-type/RF_self_training_thres85.parquet", index=False)

## KNN

In [5]:
# df_train = pd.read_csv('./data/forest-cover-type/train.csv', index_col=0 )
X_test = pd.read_csv('./data/forest-cover-type/test.csv', index_col=0 )

# # separation des données
# X = df_train.drop(columns=['Cover_Type'])
# y = df_train['Cover_Type']

# # divise les données 
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

# columns_to_keep = ['Wilderness_Area1',
#        'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
#        'Geologic_Zone_1','Climatic_Zone_1', 'Climatic_Zone_2', 'Geologic_Zone_2',
#        'Climatic_Zone_3', 'Geologic_Zone_3', 'Climatic_Zone_4',
#        'Geologic_Zone_4', 'Climatic_Zone_5', 'Geologic_Zone_5',
#        'Climatic_Zone_6', 'Geologic_Zone_6', 'Climatic_Zone_7',
#        'Geologic_Zone_7', 'Climatic_Zone_8', 'Geologic_Zone_8']

# columns_to_norm = [col for col in X.columns if col not in columns_to_keep]

# X_train_scaled = X_train.copy()
# X_val_scaled = X_val.copy()
# X_test_scaled = X_test.copy()

# scaler = StandardScaler()
# # Normalisation sur l'ensemble d'entraînement
# X_train_scaled[columns_to_norm] = scaler.fit_transform(X_train[columns_to_norm])

# # Appliquer la même transformation sur l'ensemble de validation
# X_val_scaled[columns_to_norm] = scaler.transform(X_val[columns_to_norm])

# #données test 
# X_test_scaled[columns_to_norm] = scaler.transform(X_test[columns_to_norm])

X_select = pd.read_csv("data/forest-cover-type/X_train_for_rf.csv", index_col=0)
X_select_val = pd.read_csv("data/forest-cover-type/X_val_for_rf.csv", index_col=0)
X_test_select = pd.read_csv("data/forest-cover-type/X_test_for_rf.csv", index_col=0)
y_train = pd.read_csv("data/forest-cover-type/y_train_for_rf.csv", index_col=0)
y_val = pd.read_csv("data/forest-cover-type/y_val_for_rf.csv", index_col=0)

y_training = pd.concat([y_train, y_val]).Cover_Type
print(y_training.shape)

(100000,)


In [6]:
classes = np.array([1, 2, 3, 4, 5, 6, 7])
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_training)
class_weights_dict = dict(zip(classes, class_weights))
params = {'max_depth': 50, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250}

params = {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'distance'}
knn_cls = KNeighborsClassifier(**params, n_jobs=-1)

X_semi_sup = pd.concat([X_select, X_select_val], axis=0)
y_test = pd.Series(- np.ones(X_test.shape[0]))
y_all = pd.concat([y_train.Cover_Type, y_val.Cover_Type, y_test], axis=0)

self_trainer_rf = SelfTrainingClassifier(knn_cls, threshold=0.95, max_iter=10, verbose=True)
self_trainer_rf.fit(pd.concat([X_semi_sup, X_test_select]), y_all)
predictions = self_trainer_rf.predict(X_test_select)

df_pred = pd.DataFrame(predictions, columns=["Cover_Type"]).reset_index().rename(columns={"index": "row_ID"})
display(df_pred.head(15))

df_pred.to_parquet("submission/forest-cover-type/KNN_self_training_thres95.parquet", index=False)

End of iteration 1, added 402526 new labels.
End of iteration 2, added 51258 new labels.
End of iteration 3, added 8636 new labels.
End of iteration 4, added 1606 new labels.
End of iteration 5, added 298 new labels.
End of iteration 6, added 68 new labels.
End of iteration 7, added 16 new labels.
End of iteration 8, added 5 new labels.
End of iteration 9, added 2 new labels.


,row_ID,Cover_Type
0,0,2.0
1,1,5.0
2,2,3.0
3,3,1.0
4,4,3.0
5,5,2.0
6,6,1.0
7,7,1.0
8,8,2.0
9,9,1.0
